In [1]:
import os
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.model_selection import train_test_split
import torch
import random
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification,AdamW, BertConfig, BertModel, BertPreTrainedModel
import pytorch_lightning as pl
#MODEL_NAME='bert-base-cased'
from torch import nn
import MeCab
batch=2
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tagger = MeCab.Tagger()
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [2]:
from sklearn.preprocessing import LabelEncoder


In [16]:
df = pd.read_csv('/data1/ohnishi/202271month_per_hour_geotaged_adGeocode_undersampled.csv')
index_train = int(len(df) * 0.8)
index_val = int(len(df) * 0.1)
index_test = int(len(df) * 0.1)
df = df.sample(frac=1, random_state=0)
df_train = df.iloc[0:index_train, 1:]
df_val = df.iloc[index_train : index_train + index_val, 1:]
df_test = df.iloc[
    index_train + index_val : index_train + index_val + index_test, 1:
]
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


In [4]:
df_test

,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,place_id,author_id,username,...,author_description,author_location,city_name,object,country,UserPlase_longitude,UserPlase_latitude,UserPlase_sub_longitude,UserPlase_sub_latitude,code
0,937,サントリー白州蒸溜所ツアー樽貯蔵庫に足を踏み入れた瞬間の香り…忘れられない🥃何度でも訪れたい...,2022-07-12_13:52:40,0.0,0.0,14.0,0,0fc3a3c87d946003,2795144574,molico_km,...,小学校四年生からロッテ一筋👍家族全員ロッテ大好き💕流山市在住のファン歴31年目のロッテファン...,NaN,サントリー白州蒸溜所,poi,日本,138.303687,35.828109,138.303687,35.828109,5338
1,876,ここ最近購入した目覚まし時計を譲る事となりました。中古品ですが、ナショナルクロックラジオ「簡...,2022-07-26_22:54:46,0.0,0.0,3.0,0,aa52ed3a2a57a535,1246216137559257088,UarbanLainer7,...,愛媛生まれ、奈良・静岡育ちです。奈良で障害者の支援をしてます。(発達障害を持ってます。)ST...,NaN,奈良 橿原市,city,日本,135.750495,34.463281,135.832225,34.537666,5135
2,260,ここでも35℃ #35Do https://t.co/Rp42XHmLDS,2022-07-01_10:58:09,0.0,0.0,1.0,0,fcbe3555d7b421d0,122428406,ryo1sax,...,Saxophone(Alto Tenor Soprano Baritone) player....,日本,愛知 名古屋市 熱田区,city,日本,136.888356,35.108910,136.924501,35.139906,5236
3,1174,@JirouAruma たぶんそうですよね♪店構えに出るし(笑),2022-07-30_07:53:09,0.0,0.0,2.0,0,61774861ea700565,308818748,thxeveryone,...,味の古久家公式風味アカウントです！https://t.co/Ec55dr5GYA餃子製造直売...,たいてい藤沢らへんですｗ,神奈 藤沢市,city,日本,139.394316,35.297103,139.516992,35.428977,5239
4,297,当券があるみたいなのでそろそろ難波寄ってから現場に向かいますか🚃🚃,2022-07-15_12:58:17,0.0,0.0,1.0,0,999694acfaba33dd,1035778798224998400,yukirinsan0715,...,NaN,NaN,大阪 堺市 堺区,city,日本,135.423966,34.544793,135.506079,34.608492,5135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65499,1039,@a05071211 おはようSUN🤗,2022-07-25_04:26:35,0.0,1.0,1.0,0,7fa7c66d61604cc5,2801835822,243600679m,...,至誠而不動者未之有也,広島 広島市,広島 広島市 東区,city,日本,132.464266,34.392100,132.575992,34.469456,5132
65500,1364,#仲野太賀いい俳優だよね🎵#拾われた男 #松本穂香 #要潤 #薬師丸ひろ子 #草彅剛,2022-07-10_22:53:02,0.0,0.0,1.0,0,22c3d93feadaf1c2,4720869864,ryuji25227971,...,自由奔放❗♡(^o^)v読売巨人軍⚾ジュビロ磐田⚽プロレス(三沢光晴)🏃よしもと新喜劇大好き...,NaN,静岡 浜松市 中区,city,日本,137.685262,34.679768,137.767103,34.780028,5237
65501,1403,山中湖は曇り空だったので、そんなに暑くなかったです。ハコニワさんで頂いたのは、新メニューのフ...,2022-07-02_17:54:35,0.0,0.0,6.0,0,04137ba8263d0a7a,253046034,JP_Snowleopard,...,ユキヒョウ大好き！ネコ全般が大好き！ 食べる＆料理も大好きなので全く痩せず体格はいつもデラッ...,神奈川県南足柄市,神奈 南足柄市,city,日本,139.004641,35.271780,139.137657,35.352280,5239
65502,1309,@unimaruGT @CRC03755253 通話回線付きだとだめらしいとか(´・ω・),2022-07-03_09:53:53,0.0,1.0,6.0,0,51ed8cc2a648d8af,153362336,AohataM,...,マンガ編集などー(´・ω・)つ八朔絵垢→@AohataM18,因島海賊タワー,広島 尾道市,city,日本,133.051172,34.255968,133.296953,34.572788,5133


In [17]:
y = "code"
L = LabelEncoder()
df[y] = L.fit_transform(df[y])
num_class = df[y].max() + 1  # オリジンの補正

In [6]:
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel
from transformers import BertModel
class BertForSequenceClassifier_pl(pl.LightningModule):
        
    def __init__(self, model_name, lr, num_class):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert = BertModel.from_pretrained(model_name)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_class)
        self.criterion = nn.CrossEntropyLoss()

        # BertLayerモジュールの最後を勾配計算ありに変更
        for param in self.bert.parameters():
            param.requires_grad = False
        for param in self.bert.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        preds= self.classifier(output.pooler_output)
        
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)
        #print(f"tihi is {loss}")
        return loss, preds
    
       # trainのミニバッチに対して行う処理
    def training_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        self.log('train_loss', loss)
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # validation、testでもtrain_stepと同じ処理を行う
    def validation_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    def test_step(self, batch, batch_idx):
        loss, preds = self.forward(input_ids=batch["input_ids"],
                                    attention_mask=batch["attention_mask"],
                                    labels=batch["labels"])
        return {'loss': loss,
                'batch_preds': preds,
                'batch_labels': batch["labels"]}

    # epoch終了時にvalidationのlossとaccuracyを記録
    def validation_epoch_end(self, outputs, mode="val"):
        # loss計算
        epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
        epoch_labels = torch.cat([x['batch_labels'] for x in outputs])
        epoch_loss = self.criterion(epoch_preds, epoch_labels)
        self.log(f"{mode}_loss", epoch_loss, logger=True)

        num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f"{mode}_accuracy", epoch_accuracy, logger=True)

        

    # testデータのlossとaccuracyを算出（validationの使いまわし）
    def test_epoch_end(self, outputs):
        return self.validation_epoch_end(outputs, "test")

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [7]:
best_model_path='/home/is/shuntaro-o/dev/persons_move_analysis/models/japan_model_ver2.ckpt'

In [18]:
x = "text"
df_test[x].astype(str)
df_test[x].astype(str)
df_test[x].str.replace(r'[【】]', ' ')       # 【】の除去
df_test[x].str.replace(r'[（）()]', ' ')     # （）の除去
df_test[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
df_test[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
df_test[x].str 
df_test[x].str.replace(r'https?:\/\/.*?[\r\n ]', '')  # URLの除去
df_test[x].str.replace(r'http:\/\/.*?[\r\n ]', '')  # URLの除去
df_test[x].str.replace(r'　', ' ')

/tmp/ipykernel_33438/402010954.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test[x].str.replace(r'[【】]', ' ')       # 【】の除去
/tmp/ipykernel_33438/402010954.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test[x].str.replace(r'[（）()]', ' ')     # （）の除去
/tmp/ipykernel_33438/402010954.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test[x].str.replace(r'[［］\[\]]', ' ')   # ［］の除去
/tmp/ipykernel_33438/402010954.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test[x].str.replace(r'[@＠]\w+', '')  # メンションの除去
/tmp/ipykernel_33438/402010954.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test[x].str.replace(r'https?:\/\/.*?[\r\n ]', '')  # URLの除去
/tmp/ipykernel_33438/402010954.py:10: FutureWarning: The default val

0        サントリー白州蒸溜所ツアー樽貯蔵庫に足を踏み入れた瞬間の香り…忘れられない🥃何度でも訪れたい...
1        ここ最近購入した目覚まし時計を譲る事となりました。中古品ですが、ナショナルクロックラジオ「簡...
2                    ここでも35℃ #35Do https://t.co/Rp42XHmLDS
3                         @JirouAruma たぶんそうですよね♪店構えに出るし(笑)
4                        当券があるみたいなのでそろそろ難波寄ってから現場に向かいますか🚃🚃
                               ...                        
65499                                  @a05071211 おはようSUN🤗
65500           #仲野太賀いい俳優だよね🎵#拾われた男 #松本穂香 #要潤 #薬師丸ひろ子 #草彅剛
65501    山中湖は曇り空だったので、そんなに暑くなかったです。ハコニワさんで頂いたのは、新メニューのフ...
65502        @unimaruGT @CRC03755253 通話回線付きだとだめらしいとか(´・ω・)
65503    https://t.co/1sVufWO3rlあと、もう少し！ https://t.co/E...
Name: text, Length: 65504, dtype: object

In [12]:
sentences_text_test=df_test.text.values
list_estimate = []
model = BertForSequenceClassifier_pl.load_from_checkpoint(best_model_path)
bert=model.bert.cuda()
classifier=model.classifier.cuda()
i = 0
for sentence in sentences_text_test:
    encoding = tokenizer(
    sentence,
    max_length=100,
    pad_to_max_length=True,
    truncation=True,
    return_tensors='pt')
    encoding = { k: v.cuda() for k, v in encoding.items() }
    with torch.no_grad():
        output = bert(**encoding)
        ans=classifier(output.pooler_output)
        ans = ans.to('cpu').detach().numpy().copy()
        ans = ans.argmax()
        ans = L.inverse_transform([ans])[0]
        list_estimate.append(ans)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/is/shuntaro-o/miniconda3/envs/SharedTask/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2263: Futu

In [19]:
list_estimate=pd.Series(list_estimate)
df_test=pd.concat([df_test,list_estimate],axis=1)
df_test=df_test.rename(columns={0: 'estimate_code'})

In [21]:
df_test.to_csv("/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/df_test.csv")

In [24]:
df_test

,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,place_id,author_id,username,...,author_location,city_name,object,country,UserPlase_longitude,UserPlase_latitude,UserPlase_sub_longitude,UserPlase_sub_latitude,code,estimate_code
0,937,サントリー白州蒸溜所ツアー樽貯蔵庫に足を踏み入れた瞬間の香り…忘れられない🥃何度でも訪れたい...,2022-07-12_13:52:40,0.0,0.0,14.0,0,0fc3a3c87d946003,2795144574,molico_km,...,NaN,サントリー白州蒸溜所,poi,日本,138.303687,35.828109,138.303687,35.828109,5338,5436
1,876,ここ最近購入した目覚まし時計を譲る事となりました。中古品ですが、ナショナルクロックラジオ「簡...,2022-07-26_22:54:46,0.0,0.0,3.0,0,aa52ed3a2a57a535,1246216137559257088,UarbanLainer7,...,NaN,奈良 橿原市,city,日本,135.750495,34.463281,135.832225,34.537666,5135,5236
2,260,ここでも35℃ #35Do https://t.co/Rp42XHmLDS,2022-07-01_10:58:09,0.0,0.0,1.0,0,fcbe3555d7b421d0,122428406,ryo1sax,...,日本,愛知 名古屋市 熱田区,city,日本,136.888356,35.108910,136.924501,35.139906,5236,5339
3,1174,@JirouAruma たぶんそうですよね♪店構えに出るし(笑),2022-07-30_07:53:09,0.0,0.0,2.0,0,61774861ea700565,308818748,thxeveryone,...,たいてい藤沢らへんですｗ,神奈 藤沢市,city,日本,139.394316,35.297103,139.516992,35.428977,5239,5133
4,297,当券があるみたいなのでそろそろ難波寄ってから現場に向かいますか🚃🚃,2022-07-15_12:58:17,0.0,0.0,1.0,0,999694acfaba33dd,1035778798224998400,yukirinsan0715,...,NaN,大阪 堺市 堺区,city,日本,135.423966,34.544793,135.506079,34.608492,5135,5238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65499,1039,@a05071211 おはようSUN🤗,2022-07-25_04:26:35,0.0,1.0,1.0,0,7fa7c66d61604cc5,2801835822,243600679m,...,広島 広島市,広島 広島市 東区,city,日本,132.464266,34.392100,132.575992,34.469456,5132,5439
65500,1364,#仲野太賀いい俳優だよね🎵#拾われた男 #松本穂香 #要潤 #薬師丸ひろ子 #草彅剛,2022-07-10_22:53:02,0.0,0.0,1.0,0,22c3d93feadaf1c2,4720869864,ryuji25227971,...,NaN,静岡 浜松市 中区,city,日本,137.685262,34.679768,137.767103,34.780028,5237,5135
65501,1403,山中湖は曇り空だったので、そんなに暑くなかったです。ハコニワさんで頂いたのは、新メニューのフ...,2022-07-02_17:54:35,0.0,0.0,6.0,0,04137ba8263d0a7a,253046034,JP_Snowleopard,...,神奈川県南足柄市,神奈 南足柄市,city,日本,139.004641,35.271780,139.137657,35.352280,5239,5338
65502,1309,@unimaruGT @CRC03755253 通話回線付きだとだめらしいとか(´・ω・),2022-07-03_09:53:53,0.0,1.0,6.0,0,51ed8cc2a648d8af,153362336,AohataM,...,因島海賊タワー,広島 尾道市,city,日本,133.051172,34.255968,133.296953,34.572788,5133,5135


In [40]:
import spacy
nlp = spacy.load("ja_core_news_md")

2022-11-29 19:18:26.983349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-11-29 19:18:26.983385: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


8月 DATE 0 2
東京 GPE 3 5


In [51]:
sentences = df_test.text.values
list_geo_Flag = []
for text in sentences:
    Flag=0
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_=="GPE":
            Flag=1
    if Flag==1:
        list_geo_Flag.append(1)
    else:
        list_geo_Flag.append(0)

In [56]:
list_geo_Flag=pd.Series(list_geo_Flag)
df_test=pd.concat([df_test,list_geo_Flag],axis=1)
df_test=df_test.rename(columns={0: 'Geo_Flag'})

In [57]:
df_test.to_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/df_test_adGeoflag.csv')